In [1]:
import os
import sys
import json
import time 
import copy
import argparse
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

import tqdm
import pandas as pd 
from PIL import Image
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from torchvision import transforms
from torch.utils.data import DataLoader
from models import get_model
from data_loader.LiTS import LiTSDataset
from utils.train import ObjFromDict


def load_model(run_dir, metric='validation_dice'): 
    with open(os.path.join(run_dir,'config.json')) as json_file:
        config = json.load(json_file)
    config = ObjFromDict(config)
    model = get_model(config.model)
    checkpoint_path = os.path.join(run_dir, 'best_{}.pth'.format(metric)) 
    model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
    return model, config

/home/raubyb/miniconda3/envs/MRIScoringEnv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/raubyb/miniconda3/envs/MRIScoringEnv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/raubyb/miniconda3/envs/MRIScoringEnv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
run_dir = 'runs/2020-03-27_20h9min'
model, config = load_model(run_dir)

feature_scale not specified in config, setting to default 4


/home/raubyb/LiTS/DLMI_segmentation/models/unet.py:85: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data, a=0, mode='fan_in')
/home/raubyb/LiTS/DLMI_segmentation/models/unet.py:89: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  nn.init.normal(m.weight.data, 1.0, 0.02)
/home/raubyb/LiTS/DLMI_segmentation/models/unet.py:90: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias.data, 0.0)


In [3]:
device='cpu'

In [4]:
data_path = config.dataset.root
print('data_path ', data_path)

# fix the seed for the split
split_seed = 0 
np.random.seed(split_seed)

image_dir = os.listdir(os.path.join(data_path,'Training Batch 1')) + os.listdir(os.path.join(data_path,'Training Batch 2'))
all_indexes = [ int(file_name[7:-4]) for file_name in image_dir if 'volume' in file_name]
split = np.random.permutation(all_indexes)
n_train, n_val, n_test = int(0.8 * len(split)), int(0.1 * len(split)), int(0.1 * len(split))

train = split[: n_train]
val = split[n_train : n_train+n_val]
test = split[n_train + n_val :]


# Setup Data Loader
train_dataset = LiTSDataset(data_path, train, augment=True, no_tumor=True)
val_dataset = LiTSDataset(data_path, val, no_tumor=True)
test_dataset = LiTSDataset(data_path, test, no_tumor=True)
train_dataloader = DataLoader(dataset=train_dataset, num_workers=config.dataset.num_workers, batch_size=config.training.batch_size, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, num_workers=config.dataset.num_workers, batch_size=config.training.batch_size, shuffle=False)
test_dataloader  = DataLoader(dataset=test_dataset,  num_workers=config.dataset.num_workers, batch_size=config.training.batch_size, shuffle=False)



data_path  /home/raubyb/LiTS


In [5]:
model = model.to(device)
model.eval()
val_dataloader = DataLoader(dataset=val_dataset, num_workers=config.dataset.num_workers, batch_size=1, shuffle=False)
with torch.no_grad():
    for batch_idx, (data, target) in tqdm.tqdm(enumerate(val_dataloader)):
            data = data.to(device)
            output = model(data)
            break


0it [00:00, ?it/s]

In [8]:
volume = output[0,1,:,:,:]
np.unique(volume)